# Projeto de Previsão de Falha

**Autor:** Lucas Porto  
**Desafio:** Data Science - Meli  
**Objetivo:** Documentar a arquitetura e regras de negócio do sistema de previsão de falhas


---

## Visão Geral do Projeto

### Desafio proposto

#### Descrição
Os armazéns no mercado livre possuem uma frota de máquinas que transmitem diariamente medidas sobre seu status e funcionamento.
As técnicas de manutenção preditiva são projetadas para ajudar a determinar a condição do equipamento de manutenção em serviço para prever quando a manutenção deve ser realizada.
Esta abordagem promete economia de custos em relação à manutenção preventiva de rotina ou baseada no tempo, porque as tarefas são executadas apenas quando justificada E m média uma interrupção por falha custa 4x a mais do que uma interrupção preventiva.
O arquivo "full_devices.csv" possui os valores diários de 9 atributos dos dispositivos ao longo do tempo e a coluna que você está tentando prever é chamada de 'failure' com o valor binário 0 para nenhuma falha e 1 para falha, a coluna do dispositivo possui o id do dispositivo.


### Entregável
O objetivo é gerar um notebook Jupyter com um modelo para prever a falha do dispositivo antes de uma possível falha. Tente calcular e maximizar a possível economia gerada pelo modelo.


### Realizado

Para este desafio proposto iremos criar uma regra de negócios para calcularmos o melhor custo benefício entre o custo entre uma falha não detectada e uma manuntenção desnecessária caso prevêssemos errado. Iremos explicar melhor na Regra de Negócio.

## Imports

In [1]:
from utils.data_utils import (
    load_and_explore_data,
    analyze_data_completeness,
    analyze_failure_patterns,
    verify_data_leakage,
)

from utils.feature_utils import (
    create_features,
)

from utils.model_utils import (
    temporal_cv_evaluation,
    create_pipeline_with_metadata,
)

from utils.pipeline_utils import (
    tune_neg_per_pos,
    train_evaluate_with_presplit,
    apply_optimal_thresholds,
    create_final_summary,
    save_model_and_metrics,
    analyze_feature_importance,
    calculate_event_metrics,
    select_best_model,
    execute_final_model_evaluation,
)

In [2]:
HORIZON_DAYS = 10
TAXA_ALERTAS = 0.05
C_FN = 100_000
C_FP = 25_000
COST_RATIO = C_FN / C_FP
MIN_PREC_FOR_ROI = C_FP / (C_FP + C_FN)

## Regras de Negócio

### Estrutura de Custos

A regra de negócio central é baseada na **análise de custo-benefício**:

```python
# Constantes de custo definidas no projeto
C_FN = 100_000    # Custo de uma falha não detectada (False Negative)
C_FP = 25_000     # Custo de manutenção desnecessária (False Positive)
```

### Cálculo de ROI

**ROI = (Benefícios - Custos) / Custos × 100%**

Onde:
- **Benefícios:** Falhas evitadas × Custo da falha
- **Custos:** Manutenções desnecessárias × Custo da manutenção

### Precisão Mínima e colocamos como se fosse Break-even do projeto

```python
def min_precision_for_roi(c_fn=100_000, c_fp=25_000):
    """
    Calcula a precisão mínima necessária para atingir break-even
    Fórmula: C_FP / (C_FP + C_FN) = 25k / 125k = 20%
    """
    return c_fp / (c_fp + c_fn)  # = 0.20 (20%)
```

### Ratio de Custos
```python
COST_RATIO = C_FN / C_FP  # = 4.0
# Uma falha não detectada custa 4x mais que uma manutenção desnecessária
```

### Lógica de Decisão

O sistema otimiza para:
1. **Maximizar detecção de falhas** (True Positives)
2. **Minimizar manutenções desnecessárias** (False Positives)
3. **Respeitar orçamento de alertas** (evitar muitos alertas)



## Carga dos Dados e Análise

In [3]:
# 1. Carregar e explorar dados
df = load_and_explore_data()

=== CARREGANDO DADOS ===
Shape original: (124494, 12)
Colunas: ['date', 'device', 'failure', 'attribute1', 'attribute2', 'attribute3', 'attribute4', 'attribute5', 'attribute6', 'attribute7', 'attribute8', 'attribute9']
Removendo 1 duplicatas...
Shape após limpeza: (124493, 12)


        date    device  failure  attribute1  attribute2  attribute3  \
0 2015-01-01  S1F01085        0   215630672          56           0   
1 2015-01-02  S1F01085        0     1650864          56           0   
2 2015-01-03  S1F01085        0   124017368          56           0   
3 2015-01-04  S1F01085        0   128073224          56           0   
4 2015-01-05  S1F01085        0    97393448          56           0   

   attribute4  attribute5  attribute6  attribute7  attribute8  attribute9  
0          52           6      407438           0           0           7  
1          52           6      407438           0           0           7  
2          52           6      407438           0           0    

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124493 entries, 0 to 124492
Data columns (total 12 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   date        124493 non-null  datetime64[ns]
 1   device      124493 non-null  object        
 2   failure     124493 non-null  int64         
 3   attribute1  124493 non-null  int64         
 4   attribute2  124493 non-null  int64         
 5   attribute3  124493 non-null  int64         
 6   attribute4  124493 non-null  int64         
 7   attribute5  124493 non-null  int64         
 8   attribute6  124493 non-null  int64         
 9   attribute7  124493 non-null  int64         
 10  attribute8  124493 non-null  int64         
 11  attribute9  124493 non-null  int64         
dtypes: datetime64[ns](1), int64(10), object(1)
memory usage: 11.4+ MB


### Análise de completude dos dados

In [5]:
# 2. Analisar completude dos dados
completeness_info = analyze_data_completeness(df)


=== ANÁLISE DE COMPLETUDE DOS DADOS ===
Total de registros: 124,493
Número de dispositivos únicos: 1,169
Número de datas únicas: 304
Período dos dados: 2015-01-01 00:00:00 at 2015-11-02 00:00:00
Registros esperados (todos dispositivos em todas as datas): 355,376
Registros atuais: 124,493
Diferença: 230,883 registros faltando
Taxa de completude: 35.03%
Taxa de falhas: 0.0009

Dispositivos completos: 27 (2.31%)
Datas completas: 0 (0.00%)


### Análise dos padrões de falhas

In [6]:
# 3. Analisar padres de falhas
device_failures, daily_failures = analyze_failure_patterns(df)


=== ANÁLISE DE PADRÕES DE FALHAS ===
Distribuição de falhas:
  Sem falha (0): 124,387 (99.91%)
  Com falha (1): 106 (0.09%)

Dispositivos com falhas: 106
Dispositivos sem falhas: 1063
Taxa média de falhas por dispositivo: 0.0024

Dias com falhas: 76
Dias sem falhas: 228
Taxa média de falhas por dia: 0.0009


## Feature Engineering

### Tipos de Features Criadas

- **Features Básicas**
- Atributos originais (attribute1-9)
- Identificação do dispositivo
- Timestamp da observação
- **Features Temporais**
- **Features de Contexto**
- **Features de Interação**
- **Normalização por Dispositivo**


### Resultado Final
- **198 features** criadas
- **80 features** selecionadas pelo pipeline
- **Normalização por dispositivo** para lidar com heterogeneidade


### Criação de nova target

A nova target foi pensada em nos gerar um alerta que permite detectar a falha antes que ela aconteça. Que é o objetivo do nosso trabalho.

> "Se um dispositivo falha no dia D, então nos dias D-1,..., D-10 ele deveria ter recebido um alerta."

Criamos ele com a função abaixo, isso foi pensado em criamos um alerta de até 10 dias antes do ocorrido.

```python
def make_early_warning_labels_safe(df, horizon_days=10):
```

In [7]:
# 4. Feature Engineering +  Target
df_features, new_target_col = create_features(
    df,
    target_col="failure",
    forecast_horizon=HORIZON_DAYS,
    enable_interactions=True,
    max_lags=7,
)


=== INICIANDO FEATURE ENGINEERING ===
1. Criando features básicas...
2. Criando features de dispositivo...
3. Criando features temporais...
4. Criando features de contexto...
5. Criando features de interação...
6. Criando features de tendência...
7. Otimizando tipos de dados...
8. Aplicando normalização por dispositivo...
=== NORMALIZAÇÃO POR DISPOSITIVO (rolling z-score) ===
  attribute1: normalizado (rolling=30, min_periods=5)
  attribute2: normalizado (rolling=30, min_periods=5)
  attribute3: normalizado (rolling=30, min_periods=5)
  attribute4: normalizado (rolling=30, min_periods=5)
  attribute5: normalizado (rolling=30, min_periods=5)
  attribute6: normalizado (rolling=30, min_periods=5)
  attribute7: normalizado (rolling=30, min_periods=5)
  attribute8: normalizado (rolling=30, min_periods=5)
  attribute9: normalizado (rolling=30, min_periods=5)
9. Criando target com horizonte de 10 dias...
10. Removendo vazamento de target...
  Removido 'failure' das features (vazamento de tar

In [8]:
# 5.  Verificação de data leakage
suspicious_features = verify_data_leakage(df_features, new_target_col)


=== VERIFICAÇÃO DE DATA LEAKAGE ===
Analisando 195 features contra target 'early_warn_target'
  Nenhuma correlação suspeita detectada


## Pipeline com Metadados

Quando trabalhamos com dados temporais, precisamos garantir que:
- **Metadados temporais** (datas, dispositivos) sejam preservados
- **Split temporal** seja feito antes do processamento
- **Features** sejam selecionadas apenas no treino
- **Validação** respeite a ordem cronológica


1. **Evita Data Leakage**
   - Split temporal antes do processamento
   - Features selecionadas apenas no treino
   - Metadados preservados para validação

2. **Preserva Contexto Temporal**
   - Datas mantidas para análise temporal
   - Dispositivos identificados para cooldown
   - Ordem cronológica respeitada

3. **Facilita Validação**
   - Métricas por evento
   - Análise de cooldown
   - ROI baseado em falhas reais


Separamos os dados temporais em:
- **Dados de treino**
- **Dados de validação:** separado para otimizaro o modelo
- **Dados de validação final (holdout)**: utilizado para validar sem vazamento
- **Dados de teste**: para nossa aplicação final.

In [9]:
# 6.  Pipeline com metadados
(
    X_train,
    y_train,
    X_val_model,
    y_val_model,
    X_val_hold,
    y_val_hold,
    X_test,
    y_test,
    selected_features,
    dates_train,
    devices_train,
    dates_val_model,
    devices_val_model,
    dates_val_hold,
    devices_val_hold,
    dates_test,
    devices_test,
) = create_pipeline_with_metadata(df_features, target_col=new_target_col)


=== PIPELINE SEGURO COM METADADOS ===
Split temporal:
  Train: 90,045 samples
  Val: 13,540 samples
  Test: 5,589 samples
Features disponíveis: 195
Aplicando pipeline...
Features selecionadas: 80
Validação separada:
  Modelo: 2,708 samples
  Calibração: 10,832 samples


## Validação Cruzada Temporal

A **Validação Cruzada Temporal** é uma técnica que simula o cenário real de produção, onde:
- **Dados futuros** não estão disponíveis no treinamento
- **Ordem cronológica** deve ser respeitada
- **Performance** deve ser consistente ao longo do tempo

### Implementação de proteção

1. **Purge de 7 dias** entre treino e validação
2. **Split temporal rigoroso** (não aleatório)
3. **Pipeline aplicado** em cada fold
4. **Métricas consistentes** entre folds

### Sinais de Alerta

- **Custo crescente** ao longo dos folds
- **Recall decrescente** no tempo
- **Alta variabilidade** entre folds
- **Performance ruim** em folds recentes

In [10]:
# 7. CV Ttemporal
cv_output = temporal_cv_evaluation(
    df_features,
    target_col=new_target_col,
    n_splits=3,
    purge_days=7,
)
cv_results = cv_output["cv_results"]
cv_summary = cv_output["cv_summary"]


=== ROLLING ORIGIN CV (n=3) COM PURGE ===
Fold 1: cost=392.0 | AP=0.0043 | recall=0.091
Fold 2: cost=538.0 | AP=0.0777 | recall=0.574
Fold 3: cost=970.0 | AP=0.2122 | recall=0.459
Médias: {'fold': 2.0, 'cost': 633.3333333333334, 'ap': 0.09806541993735318, 'recall': 0.37489316719895643}

Pipeline:
  Train: 90,045 samples (0.0094 failure rate)
  Val: 13,540 samples (0.0127 failure rate)
  Features selecionadas: 80


Com base nos nossos dados, temos um modelo com custo aumentando com o tempo, variabilizade alta, mas o recall se mantem um valor razoável.

Podemos ver que temos drift nos dados e alta variabilidade no tempo devido nosso custo aumentar.

## Otimizando o Balanceamento

### Criação de parâmetro para otimização

O **neg_per_pos** é um parâmetro criado que define quantos **exemplos negativos** (não-falha) usar para cada **exemplo positivo** (falha) durante o treinamento:

```python
# Exemplo:
# neg_per_pos = 8 significa:
# - 1 exemplo de falha (positivo)
# - 8 exemplos de não-falha (negativos)
# - Ratio: 1:8 (12.5% de falhas)
```

### Por Que Precisamos de Tuning?

#### **1. Problema de Classes Desbalanceadas**
- **Dados originais**: 0.09% de falhas (106 falhas em 124,494 registros)
- **Após early warning**: ~1% de falhas (ainda muito desbalanceado)
- **Modelo precisa** de mais exemplos negativos para aprender

#### **2. Trade-off Crítico**
- **Muitos negativos** → Modelo fica conservador (baixo recall)
- **Poucos negativos** → Modelo fica agressivo (muitos falsos positivos)
- **Balanceamento ótimo** → Performance máxima


### Critérios de Seleção

#### **1. Custo de Negócio**
- **Custo total** = 4 × FN + 1 × FP
- **Objetivo**: Minimizar custo
- **Prioridade**: Evitar falhas não detectadas

#### **2. Taxa de Alertas**
- **Máximo**: 5% dos dispositivos
- **Objetivo**: Não sobrecarregar operações
- **Balanceamento**: Performance vs. Operacional

#### **3. Métricas Técnicas**
- **Precision**: > 20% (break-even)
- **Recall**: > 60% (cobertura de falhas)
- **F1-Score**: Balanceamento geral



In [11]:
# 8. Otimização e Balanceamento
best_us = tune_neg_per_pos(
    X_train,
    y_train,
    X_val_model,
    y_val_model,
    candidates=(6, 8, 10, 12, 15, 20),
    max_alert_rate=TAXA_ALERTAS,
)


=== TUNING NEG_PER_POS ===
Testando neg_per_pos=6...

=== APLICANDO UNDERSAMPLING ===
Dataset original: 90045 amostras | dist: {0: np.int64(89197), 1: np.int64(848)} | rate=0.0094
US: 90045 -> 5936 | pos=848 neg=5088 (neg/pos~6.0, antes~105.2)
  neg_per_pos=6: custo=232, thr=0.869, alertas=0.5%, precisão=71.4%, método=cost_optimal
Testando neg_per_pos=8...

=== APLICANDO UNDERSAMPLING ===
Dataset original: 90045 amostras | dist: {0: np.int64(89197), 1: np.int64(848)} | rate=0.0094
US: 90045 -> 7632 | pos=848 neg=6784 (neg/pos~8.0, antes~105.2)
  neg_per_pos=8: custo=204, thr=0.349, alertas=5.0%, precisão=29.4%, método=budget_fallback
Testando neg_per_pos=10...

=== APLICANDO UNDERSAMPLING ===
Dataset original: 90045 amostras | dist: {0: np.int64(89197), 1: np.int64(848)} | rate=0.0094
US: 90045 -> 9328 | pos=848 neg=8480 (neg/pos~10.0, antes~105.2)
  neg_per_pos=10: custo=237, thr=0.902, alertas=0.5%, precisão=64.3%, método=cost_optimal
Testando neg_per_pos=12...

=== APLICANDO UNDERS

Dentro dos parâmetros que selecionamos, tivemos o melhor tunning com: 
- Distribuição: Negativos x Positivos em 8

## Treinando os Modelos

### **Modelos Selecionados**
- XGBoost: XGBClassifier
- LGBM: LGBMClassifier
- Ensemble entre ambos

### **Otimização de Thresholds**

Utilizamos dentro do treinamento a seleção do melhor threshold.


| Métrica Técnica | Métrica de Negócio | Implementação |
|-----------------|-------------------|----------------|
| **AUC-ROC** | Capacidade de discriminação | `roc_auc_score()` |
| **Average Precision** | Performance em desbalanceado | `average_precision_score()` |
| **Confusion Matrix** | **Custo (4×FN + 1×FP)** | `4 * fn + fp` |
| **Threshold** | **Orçamento de alertas** | `tune_alert_budget_simple()` |
| **Precision** | **Break-even (20%)** | `min_precision=0.20` |

In [12]:
# 9. Treinar e avaliar com conjuntos pr-split
models_results = train_evaluate_with_presplit(
    X_train,
    y_train,
    X_val_model,
    y_val_model,
    X_test,
    y_test,
    min_pr=MIN_PREC_FOR_ROI,
    X_val_hold=X_val_hold,
    y_val_hold=y_val_hold,
    dates_train=dates_train,
    devices_train=devices_train,
    dates_val=dates_val_model,
    devices_val=devices_val_model,
    dates_test=dates_test,
    devices_test=devices_test,
    random_state=42,
    apply_balance=True,
    neg_per_pos=best_us["neg_per_pos"],
    max_neg_cap=50_000,
    calibrate=True,
    use_alert_budget=True,
    alert_budgets=(0.01, 0.015, 0.02, 0.03),
    feature_names=selected_features,
    selected_features=selected_features,
)


=== TREINANDO MODELOS COM CONJUNTOS PR-SPLIT (BUDGET+CALIBRAÇÃO) ===

=== UNDERSAMPLING HARD NEGATIVES ===
Undersampling hard negatives aplicado: 7632 amostras
Scale pos weight: 1.00
Train: 7,632 | Val: 2,708 | Test: 5,589

=== TREINANDO XGBOOST ===
Calibrao time-aware aplicada (XGB).
Tuner: Budget...
[XGB] fallback - melhor precisão disponível (thr=0.298, alertas=0.3%)
XGB | AUC=0.7660 AP=0.1226 thr=0.298 cost=116.0 | CM=[[5552, 4], [28, 5]]

=== TREINANDO LIGHTGBM ===
Calibrao time-aware aplicada (LGB).
Tuner: Budget...
[LGB] fallback - melhor precisão disponível (thr=0.350, alertas=1.7%)
LGB | AUC=0.8917 AP=0.0900 thr=0.350 cost=132.0 | CM=[[5556, 0], [33, 0]]
Ensemble ponderado: XGB=0.464, LGB=0.536
[budget tuner] best_alert_rate=1.0% | thr=0.226 | val_cost=294.0 | cm=(np.int64(2611), np.int64(30), np.int64(66), np.int64(1))
Ensemble | AUC=0.9017 AP=0.2301 thr=0.226 cost=116.0 | CM=[[5552, 4], [28, 5]]
Calculando predições de holdout durante o treinamento...


## Aplicação dos Thresholds

A função `apply_optimal_thresholds` é a **otimização final** do sistema que:

1. **Testa múltiplos orçamentos** de alertas (0.2% a 5%)
2. **Encontra thresholds ótimos** para cada orçamento
3. **Avalia performance** com métricas de negócio
4. **Escolhe a melhor configuração** para produção

### Processo de Otimização

#### Teste em orçamentos
```python
for budget in budgets:
    # Encontrar threshold que respeita o orçamento
    threshold = find_threshold_for_budget(
        y_val, 
        model_proba_val,
        target_budget=budget
    )
    
    # Aplicar threshold no teste
    y_pred_test = (model_proba_test >= threshold).astype(int)
    
    # Calcular métricas
    cost = calculate_business_cost(y_test, y_pred_test)
    precision = precision_score(y_test, y_pred_test)
    recall = recall_score(y_test, y_pred_test)
    alert_rate = y_pred_test.mean()
    
    # Verificar restrições
    if precision >= min_precision and alert_rate <= budget:
        results[budget] = {
            'threshold': threshold,
            'cost': cost,
            'precision': precision,
            'recall': recall,
            'alert_rate': alert_rate
        }
```

#### Seleção da melhor configuração
```python
# Escolher orçamento com menor custo
best_budget = min(results.keys(), key=lambda x: results[x]['cost'])

# Configuração final
best_config = {
    'budget': best_budget,
    'threshold': results[best_budget]['threshold'],
    'cost': results[best_budget]['cost'],
    'precision': results[best_budget]['precision'],
    'recall': results[best_budget]['recall'],
    'alert_rate': results[best_budget]['alert_rate']
}
```


In [13]:
# 10. Aplicação dos thresholds ótimos com orçamento de alertas
models_results = apply_optimal_thresholds(
    models_results,
    y_val_model,
    y_test,
    dates_val=dates_val_model,
    devices_val=devices_val_model,
    dates_test=dates_test,
    devices_test=devices_test,
    df_features=df,
    min_precision=MIN_PREC_FOR_ROI,
    c_fn=C_FN,
    c_fp=C_FP,
    budgets=(
        0.002,
        0.003,
        0.004,
        0.005,
        0.006,
        0.008,
        0.010,
        0.012,
        0.015,
        0.020,
        0.030,
        0.050,
    ),
    horizon_days=HORIZON_DAYS,
)


APLICAÇÃO DOS THRESHOLDS ÓTIMOS (min_precision=20%)
Tuner: Budget...

XGBoost: thr=0.3000 | budget~1.000% | alertas=0.16% | precisão_test=55.6% | custo=116 | prec_event=20.0% | recall_event=33.3% | ROI=0.0%
Tuner: Budget...

LightGBM: thr=0.3500 | budget~1.000% | alertas=0.00% | precisão_test=0.0% | custo=132 | prec_event=0.0% | recall_event=0.0% | ROI=0.0%
Tuner: Budget...

EnsembleAvg: thr=0.2300 | budget~1.000% | alertas=0.16% | precisão_test=55.6% | custo=116 | prec_event=20.0% | recall_event=33.3% | ROI=0.0%


## Feature Importance

Analisamos o feature importance para cada modelo, para analisarmos quais são mais improtantes.

In [14]:
# 11. Analisar feature importance
models_results = analyze_feature_importance(models_results, selected_features)


=== ANÁLISE DE FEATURE IMPORTANCE ===

XGBoost - Top 20 Features:
0.1372 - attribute4_normalized
0.0754 - attribute7_ma_3
0.0680 - attribute7
0.0361 - attribute4_slope_7
0.0347 - attribute2_attribute4_sum
0.0343 - attribute2_std_14
0.0337 - attribute2_lag_2
0.0311 - attribute6_lag_3
0.0296 - attribute2_lag_3
0.0271 - attribute6_ma_14
0.0239 - attribute9_lag_7
0.0217 - doy_cos
0.0203 - attribute2_slope_14
0.0181 - attribute6_ma_3
0.0165 - attribute4_std_expanding
0.0163 - attribute6_lag_2
0.0158 - attribute6_ma_7
0.0153 - attribute5_lag_1
0.0147 - attribute2
0.0143 - attribute6_lag_1

LightGBM - Top 20 Features:
2.0000 - doy_sin
2.0000 - attribute6_lag_1
2.0000 - attribute6_lag_2
2.0000 - attribute1_std_14
1.0000 - attribute2_std_14
1.0000 - attribute7
1.0000 - week_sin
1.0000 - attribute6_lag_7
1.0000 - device_obs_so_far
1.0000 - attribute5_normalized
1.0000 - attribute6_normalized
1.0000 - attribute2_attribute4_sum
1.0000 - attribute4_slope_7
0.0000 - attribute6
0.0000 - attribute5
0

## Calculando métricas por evento

A métrica por evento é calculada para ser mais um dos parametros usados na seleção do nosso modelo.

In [15]:
# 12. Métricas por evento
models_results = calculate_event_metrics(models_results, df, dates_test, devices_test)


=== MÉTRICAS POR EVENTO ===
XGBoost | event_coverage=33.3% | alerts_per_100dev_week=0.79
LightGBM | event_coverage=0.0% | alerts_per_100dev_week=0.00
EnsembleAvg | event_coverage=33.3% | alerts_per_100dev_week=0.79


In [16]:
# 13. Selecionar o melhor modelo usando holdout e métrica de custo
best_model_name, best_model_results, comparison_df = select_best_model(
    models_results, y_val_hold
)


SELEÇÃO DO MELHOR MODELO (VALIDAÇÃO HOLDOUT)

COMPARAÇÃO DE MODELOS:
        Modelo  AUC-ROC  Custo  Precision  Recall
0      XGBoost   0.7660    116     0.3158  0.0571
2  EnsembleAvg   0.9017    116     0.3158  0.0571
1     LightGBM   0.8917    132     0.0000  0.0000

MELHOR MODELO SELECIONADO: XGBoost
   Custo: 116


In [17]:
# 14. Execução final do melhor modelo com dados de teste
final_test_results = execute_final_model_evaluation(
    best_model_name=best_model_name,
    best_model_results=best_model_results,
    X_test=X_test,
    y_test=y_test,
    dates_test=dates_test,
    devices_test=devices_test,
    df=df,
    c_fn=C_FN,
    c_fp=C_FP,
    horizon_days=HORIZON_DAYS,
)


EXECUÇÃO FINAL DO MELHOR MODELO: XGBoost
Executando XGBoost...
Aplicando calibração...

RESULTADOS DO TESTE FINAL:
   Modelo: XGBoost
   Threshold: 0.3000
   Alert Rate: 0.002
   Recall: 0.152
   AUC-PR: 0.123

MÉTRICAS DE NEGÓCIO:
   ROI por Evento: 0.0%
   Precisão por Evento: 20.0%
   Recall por Evento: 33.3%

ANÁLISE DE ALERTAS:
   Total de Alertas: 9
   Taxa de Alertas: 0.16%
   Falsos Negativos: 28
   Falsos Positivos: 4


In [18]:
# 15. Resumo final e métricas de ROI
summary = create_final_summary(
    models_results,
    dates_train,
    dates_val_model,
    dates_val_hold,
    dates_test,
    selected_features,
    cv_results,
    cv_summary,
    suspicious_features,
    c_fn=C_FN,
    c_fp=C_FP,
    min_roi=MIN_PREC_FOR_ROI,
)

save_paths = save_model_and_metrics(
    best_model_name=best_model_name,
    best_model_results=best_model_results,
    selected_features=selected_features,
    summary=summary,
    final_test_results=final_test_results,
    cv_results=cv_results,
    cv_summary=cv_summary,
    suspicious_features=suspicious_features,
    model_name="device_failure_model",
    save_dir="models",
)


RESUMO FINAL - MÉTRICAS ESSENCIAIS

COMPARAÇÃO DE MODELOS:
--------------------------------------------------
XGBoost:
  ROI por Evento: 0.0%
  Precisão por Evento: 20.0%
  Recall por Evento: 33.3%
  Custo: 116

LightGBM:
  ROI por Evento: 0.0%
  Precisão por Evento: 0.0%
  Recall por Evento: 0.0%
  Custo: 132

EnsembleAvg:
  ROI por Evento: 0.0%
  Precisão por Evento: 20.0%
  Recall por Evento: 33.3%
  Custo: 116

ANÁLISE DE BREAK-EVEN:
------------------------------
Precisão mínima para break-even: 20.0%
Ratio custo FN/FP: 4.0

XGBoost:
  Precisão: 20.0% [OK]
  ROI: 0.0%

LightGBM:
  Precisão: 0.0% [FAIL]
  ROI: 0.0%

EnsembleAvg:
  Precisão: 20.0% [OK]
  ROI: 0.0%


SALVANDO MODELO E MÉTRICAS - 20251006_100036
Melhor modelo salvo: models\device_failure_model_xgboost_20251006_100036.pkl
Métricas de performance salvas: models\device_failure_model_metrics_20251006_100036.json
Resumo executivo salvo: models\device_failure_model_executive_20251006_100036.json

Melhor modelo e métricas s